In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
train_data = pd.read_csv("../data_acquisition/data_0.3/data_game_values_train.csv", sep=";")

In [ ]:
xg_mean = np.mean(np.mean(train_data["home_xG"]) + np.mean(train_data["away_xG"]))
xg_against_mean = np.mean(np.mean(train_data["home_xg_against"]) + np.mean(train_data["away_xg_against"]))

xt_mean = np.mean(np.mean(train_data["home_xT_all"]) + np.mean(train_data["away_xT_all"]))
xt_against_mean = np.mean(np.mean(train_data["home_xt_all_against"]) + np.mean(train_data["away_xt_all_against"]))

form_for_mean = np.mean(np.mean(train_data["ha_form_home_for"] / 5) + np.mean(train_data["ha_form_away_for"] / 5))
form_against_mean = np.mean(np.mean(train_data["ha_form_home_against"] / 5) + np.mean(train_data["ha_form_away_against"] / 5))

In [ ]:
features = np.swapaxes(np.array([train_data["home_xG"] - xg_mean, train_data["away_xg_against"] - xg_against_mean,
                                train_data["away_xG"] - xg_mean, train_data["home_xg_against"] - xg_against_mean,
                                train_data["home_xT_all"] - xt_mean, train_data["away_xt_all_against"] - xt_against_mean,
                                train_data["away_xT_all"] - xt_mean, train_data["home_xt_all_against"] - xt_against_mean]), 0, 1)

# gi = np.swapaxes(np.array([train_data["home_gi"], train_data["away_gi"]]), 0, 1)

form = np.swapaxes(np.array([(train_data["ha_form_home_for"] / 5) - form_for_mean, (train_data["ha_form_home_against"] / 5) - form_against_mean,
                            (train_data["ha_form_away_for"] / 5) - form_for_mean, (train_data["ha_form_away_against"] / 5) - form_against_mean]), 0, 1)

goals = np.swapaxes(np.array([train_data["home_score"], train_data["away_score"]]), 0, 1)

elo_diff = np.swapaxes(np.array([(train_data["elo_home"] / 1000) - (train_data["elo_away"] / 1000)]), 0, 1)
       

In [ ]:
goals_clas = [np.array([1, 0, 0]) if x[0] > x[1] else np.array([0, 1, 0]) if x[0] == x[1] else np.array([0, 0, 1]) for x in goals]

In [ ]:
X = np.concatenate([features, form, elo_diff], axis=1)
y = np.array(goals_clas)

In [ ]:
model = Sequential()
model.add(Dense(26, input_shape=(13,), activation='relu'))
model.add(Dense(52, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(3, activation="sigmoid"))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mean_squared_error'])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
test_data = pd.read_csv("../data_acquisition/data_0.3/data_game_values_test_odds.csv", sep=";").dropna()

features = np.swapaxes(np.array([test_data["home_xG"] - xg_mean, test_data["away_xg_against"] - xg_against_mean,
                                test_data["away_xG"] - xg_mean, test_data["home_xg_against"] - xg_against_mean,
                                test_data["home_xT_all"] - xt_mean, test_data["away_xT_all"] - xt_mean,
                                test_data["home_xt_all_against"] - xt_against_mean, test_data["away_xt_all_against"] - xt_against_mean]), 0, 1)
# features_a = np.swapaxes(np.array([train_data["away_xG"], train_data["away_xT_only_pos"], train_data["home_xg_against"], train_data["home_xt_only_pos_against"]]), 0, 1)

gi = np.swapaxes(np.array([test_data["home_gi"], test_data["away_gi"]]), 0, 1)
# gi_a = np.swapaxes(np.array([train_data["away_gi"]]), 0, 1)

form = np.swapaxes(np.array([(test_data["ha_form_home_for"] / 5) - form_for_mean, (test_data["ha_form_home_against"] / 5) - form_against_mean,
                            (test_data["ha_form_away_for"] / 5) - form_for_mean, (test_data["ha_form_away_against"] / 5) - form_against_mean]), 0, 1)

goals = np.swapaxes(np.array([test_data["home_score"], test_data["away_score"]]), 0, 1)

elo = np.swapaxes(np.array([(test_data["elo_home"] / 1000) - (test_data["elo_away"] / 1000)]), 0, 1)


In [ ]:
goals_clas = [np.array([1, 0, 0]) if x[0] > x[1] else np.array([0, 1, 0]) if x[0] == x[1] else np.array([0, 0, 1]) for x in goals]

In [ ]:
X_test = np.concatenate([features, form, elo], axis=1)
y_test = np.array(goals_clas)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# pred, act = [], []
# for idx, i in enumerate(np.round(predictions)):
#     if i[0] > i[1]:
#         pred.append(0)
#     if i[0] == i[1]:
#         pred.append(1)
#     if i[0] < i[1]:
#         pred.append(2)
#     if y_test[idx][0] > y_test[idx][1]:
#         act.append(0)
#     if y_test[idx][0] == y_test[idx][1]:
#         act.append(1)
#     if y_test[idx][0] < y_test[idx][1]:
#         act.append(2)
    
# df = pd.DataFrame({"act": act, "pred": pred})

In [ ]:
pred, act = [], []
for idx, i in enumerate(np.round(predictions)):
    pred.append(np.argmax(i))
    if y_test[idx][0] > y_test[idx][1]:
        act.append(0)
    if y_test[idx][0] == y_test[idx][1]:
        act.append(1)
    if y_test[idx][0] < y_test[idx][1]:
        act.append(2)
    
df = pd.DataFrame({"act": act, "pred": pred})

In [ ]:
np.sum(df[df["act"] == df["pred"]]) / df.shape[0]

In [ ]:
df